In [9]:
# --- Carga de Datos ---
import pandas as pd
df = pd.read_csv('trainData.csv')
df.head()

,Unnamed: 0,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,...,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
0,55124,2,0,0,0,0,0,0,0,0,...,1,4,5,43200,0,0,0,0,0,0
1,44575,3,0,0,0,0,0,0,0,0,...,1,2,5,14399,1,1,0,0,0,0
2,87793,2,0,0,0,0,0,0,0,0,...,1,4,0,292,1,0,0,0,0,0
3,5689,2,0,0,0,0,0,0,0,0,...,1,3,1,3600,1,1,0,0,0,0
4,38932,2,0,0,0,0,0,0,0,0,...,1,2,1,21596,1,2,0,0,0,0


In [10]:
# --- Preprocesamiento: Verificar valores nulos y valores negativos ---
import numpy as np

# Reemplazar valores nulos por 0
df = df.fillna(0)

# Reemplazar valores negativos por 0 en todas las columnas numéricas
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].clip(lower=0)

Ahora viene el calculo de los mejores hiperparametros. Como en algunos casos esto se demora mucho (50min), dejamos el codigo que lo genera en comillas para que pueda seguir ejecutandose las partes de mas abajo. Lo unico que hay que rescatar de las partes en comillas son los resultados obtenidos.

In [ ]:




# --- Modelamiento y Evaluación ---
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  

# Separar features y target
X = df.drop('phishing', axis=1)  
y = df['phishing'] 

# Definir métrica
scoring = 'f1'  # Usar F1-Score como métrica de evaluación


"""
# 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  



# Normalización y balanceo dentro de cada fold

knn_params = {'knn__n_neighbors': list(range(1, 21))}
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('knn', KNeighborsClassifier())
])

grid_knn = GridSearchCV(knn_pipeline, knn_params, cv=cv, scoring=scoring, n_jobs=-1)
grid_knn.fit(X, y)
print("Mejor K para KNN:", grid_knn.best_params_)
print("Mejor F1-Score:", grid_knn.best_score_)

# Esto se demora mucho en ejecutarse.(5min aprox)

# --- Resultado obtenido  ---
# Mejor K para KNN: {'knn__n_neighbors': 4} 
# Mejor F1-Score: 0.9215568833373402
"""

'\n# 10-fold cross-validation\ncv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  \n\n\n\n# Normalización y balanceo dentro de cada fold\n\nknn_params = {\'knn__n_neighbors\': list(range(1, 21))}\nknn_pipeline = Pipeline([\n    (\'scaler\', StandardScaler()),\n    (\'smote\', SMOTE(random_state=42)),\n    (\'knn\', KNeighborsClassifier())\n])\n\ngrid_knn = GridSearchCV(knn_pipeline, knn_params, cv=cv, scoring=scoring, n_jobs=-1)\ngrid_knn.fit(X, y)\nprint("Mejor K para KNN:", grid_knn.best_params_)\nprint("Mejor F1-Score:", grid_knn.best_score_)\n\n# Esto se demora mucho en ejecutarse.(5min aprox)\n\n# --- Resultado obtenido  ---\n# Mejor K para KNN: {\'knn__n_neighbors\': 4} \n# Mejor F1-Score: 0.9215568833373402\n'

In [12]:
"""

# --- Árbol de Decisión ---
tree_params = {'tree__max_depth': list(range(1, 21))}
tree_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('tree', DecisionTreeClassifier(random_state=42))
])

grid_tree = GridSearchCV(tree_pipeline, tree_params, cv=cv, scoring=scoring, n_jobs=-1)
grid_tree.fit(X, y)
print("Mejor max_depth para Árbol:", grid_tree.best_params_)
print("Mejor F1-Score:", grid_tree.best_score_)

# --- Resultado obtenido ---
#   Mejor max_depth para Árbol: {'tree__max_depth': 14}
#   Mejor F1-Score: 0.935073970535818

"""

'\n\n# --- Árbol de Decisión ---\ntree_params = {\'tree__max_depth\': list(range(1, 21))}\ntree_pipeline = Pipeline([\n    (\'scaler\', StandardScaler()),\n    (\'smote\', SMOTE(random_state=42)),\n    (\'tree\', DecisionTreeClassifier(random_state=42))\n])\n\ngrid_tree = GridSearchCV(tree_pipeline, tree_params, cv=cv, scoring=scoring, n_jobs=-1)\ngrid_tree.fit(X, y)\nprint("Mejor max_depth para Árbol:", grid_tree.best_params_)\nprint("Mejor F1-Score:", grid_tree.best_score_)\n\n# --- Resultado obtenido ---\n#   Mejor max_depth para Árbol: {\'tree__max_depth\': 14}\n#   Mejor F1-Score: 0.935073970535818\n\n'

In [13]:
"""
# --- Naive Bayes ---
nb_params = {'nb__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]}
nb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('nb', GaussianNB())
])

grid_nb = GridSearchCV(nb_pipeline, nb_params, cv=cv, scoring=scoring, n_jobs=-1)
grid_nb.fit(X, y)
print("Mejor var_smoothing para NB:", grid_nb.best_params_)
print("Mejor F1-Score:", grid_nb.best_score_)

# --- Resultado Obtenido ---
# Mejor var_smoothing para NB: {'nb__var_smoothing': 0.01}
# Mejor F1-Score: 0.4459903427029562

"""

'\n# --- Naive Bayes ---\nnb_params = {\'nb__var_smoothing\': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]}\nnb_pipeline = Pipeline([\n    (\'scaler\', StandardScaler()),\n    (\'smote\', SMOTE(random_state=42)),\n    (\'nb\', GaussianNB())\n])\n\ngrid_nb = GridSearchCV(nb_pipeline, nb_params, cv=cv, scoring=scoring, n_jobs=-1)\ngrid_nb.fit(X, y)\nprint("Mejor var_smoothing para NB:", grid_nb.best_params_)\nprint("Mejor F1-Score:", grid_nb.best_score_)\n\n# --- Resultado Obtenido ---\n# Mejor var_smoothing para NB: {\'nb__var_smoothing\': 0.01}\n# Mejor F1-Score: 0.4459903427029562\n\n'

In [14]:
""""

# --- Regresión Logística: búsqueda del mejor penalty ---


log_params = {
    'log__penalty': ['l1', 'l2', 'elasticnet'],
    'log__solver': ['saga'],
    'log__C': [0.01, 0.1, 1, 10],
    'log__l1_ratio': [0.5]  # Solo se usa cuando penalty='elasticnet'
}
log_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('log', LogisticRegression(max_iter=1000, random_state=42))
])

grid_log = GridSearchCV(log_pipeline, log_params, cv=cv, scoring=scoring, n_jobs=-1, verbose=2)
grid_log.fit(X, y)
print("Mejor penalty para Regresión Logística:", grid_log.best_params_['log__penalty'])
print("Mejor combinación de hiperparámetros:", grid_log.best_params_)
print("Mejor F1-Score:", grid_log.best_score_)


#   -- Resultados Obtenidos-- (La ejecucion tomo 50min)
#   Mejor penalty para Regresión Logística: l1
#   Mejor combinación de hiperparámetros: {'log__C': 1, 'log__l1_ratio': 0.5, 'log__penalty': 'l1', 'log__solver': 'saga'}
#   Mejor F1-Score: 0.9040002993680263
"""

'"\n\n# --- Regresión Logística: búsqueda del mejor penalty ---\n\n\nlog_params = {\n    \'log__penalty\': [\'l1\', \'l2\', \'elasticnet\'],\n    \'log__solver\': [\'saga\'],\n    \'log__C\': [0.01, 0.1, 1, 10],\n    \'log__l1_ratio\': [0.5]  # Solo se usa cuando penalty=\'elasticnet\'\n}\nlog_pipeline = Pipeline([\n    (\'scaler\', StandardScaler()),\n    (\'smote\', SMOTE(random_state=42)),\n    (\'log\', LogisticRegression(max_iter=1000, random_state=42))\n])\n\ngrid_log = GridSearchCV(log_pipeline, log_params, cv=cv, scoring=scoring, n_jobs=-1, verbose=2)\ngrid_log.fit(X, y)\nprint("Mejor penalty para Regresión Logística:", grid_log.best_params_[\'log__penalty\'])\nprint("Mejor combinación de hiperparámetros:", grid_log.best_params_)\nprint("Mejor F1-Score:", grid_log.best_score_)\n\n\n#   -- Resultados Obtenidos-- (La ejecucion tomo 50min)\n#   Mejor penalty para Regresión Logística: l1\n#   Mejor combinación de hiperparámetros: {\'log__C\': 1, \'log__l1_ratio\': 0.5, \'log__penal

Luego de comparar los f1-score de cada modelo con su mejor hiperparametro, determinamos que el mejor model es el arbol de decision con profundidad 13. Este dio un f1-score de 0.935073970535818 que fue el mas alto de todos los modelos. Ahora entrenamos este modelo con los datos de entrenamiento, para luego aplicarlos al de testeo.

In [16]:
# --- Entrenamiento final con Árbol de Decisión y evaluación en set de testeo ---

from sklearn.model_selection import train_test_split

# Separar en entrenamiento y testeo (por ejemplo, 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Usar el mejor hiperparámetro encontrado
mejor_max_depth = 13  # Cambia este valor si tu resultado fue otro

# Pipeline final
final_tree_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('tree', DecisionTreeClassifier(max_depth=mejor_max_depth, random_state=42))
])

# Entrenar solo con el set de entrenamiento
final_tree_pipeline.fit(X_train, y_train)

# Predecir en el set de testeo
y_pred = final_tree_pipeline.predict(X_test)

# Evaluar desempeño
from sklearn.metrics import classification_report, confusion_matrix

print("Reporte de clasificación en el set de testeo:")
print(classification_report(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

Reporte de clasificación en el set de testeo:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      9278
           1       0.92      0.95      0.94      4906

    accuracy                           0.95     14184
   macro avg       0.95      0.95      0.95     14184
weighted avg       0.96      0.95      0.95     14184

Matriz de confusión:
[[8863  415]
 [ 230 4676]]
